In [2]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0,
    groq_api_key='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',
    timeout=None,
    max_retries=2,
    # other params...
)
response=llm.invoke("The first president of India was?")
print(response.content)

The first president of India was Dr. Rajendra Prasad. He served as the president from January 26, 1950, to May 13, 1962.


In [11]:
from langchain_community.document_loaders import WebBaseLoader
loader=WebBaseLoader("https://www.maersk.com/careers/vacancies/wd/Software-Engineer---Python-PySpark-Data-Engineer---Energy-Transition--Full-Remote-available-for-XP_R125543-1/jt-software-engineer-python-pyspark-data-engineer")
page_data=loader.load().pop().page_content
print(page_data)




Software Engineer - Python/PySpark/Data Engineer vacancy | R125543 | Maersk


 
              Home
             
              Careers
             
              Vacancies
             Share  
      LinkedIn
     
      Facebook
     
      Twitter
     
      Email
     Copy link 
      WeChat
     Software Engineer - Python/PySpark/Data Engineer 
      Full time
     
      IN - Bangalore, More...
     
      Posted 23 Days Ago
     Apply now
       Opportunity The world is facing a climate emergency. At Maersk we see it as our obligation – as an industry leader in logistics and with the resources available to us – to do all we can to get to net zero operations as fast as possible and to help our customers decarbonize their global supply chains end to end.As a key player in an industry responsible for 3.5 billion tons of annual direct emissions, A.P. Moller - Maersk (Maersk) is committed to leading the decarbonization of logistics and taking responsibility to be part of the solut

In [12]:
from langchain_core.prompts import PromptTemplate
prompt_extract=PromptTemplate.from_template(
        """ SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ###INSTRUCTION:
        The scraped text is from the career's page of website.
        Your job is to extract the job posting and return them in JSON format containing the following key:
        'role','experience','skills', and 'description'.
        Only return the valid JSON inside,remove all data outside the json object
        Extract only first object
        ###VALID JSON(NO PREAMBLE):
    
            """
)
chain_extract=prompt_extract|llm
res=chain_extract.invoke(input={'page_data':page_data})
print(res.content)

```
{
  "role": "Software Engineer - Python/PySpark/Data Engineer",
  "experience": "Not specified",
  "skills": [
    "Python",
    "PySpark",
    "Great Expectations",
    "Data Plane/ Pipe Dream",
    "Elixir",
    "Erlang",
    "Ruby",
    "Functional programming",
    "Extreme programming practices",
    "Test Driven Design"
  ],
  "description": "The world is facing a climate emergency. At Maersk we see it as our obligation – as an industry leader in logistics and with the resources available to us – to do all we can to get to net zero operations as fast as possible and to help our customers decarbonize their global supply chains end to end. Our Energy Transition technology team is at the heart of this work, using Advanced Technology and Development Techniques to rapidly develop and continuously deploy the systems we need to identify, track and reduce our emissions."
}
```


In [13]:
from langchain_core.output_parsers import JsonOutputParser
json_parser=JsonOutputParser()
json_res=json_parser.parse(res.content)
json_res

{'role': 'Software Engineer - Python/PySpark/Data Engineer',
 'experience': 'Not specified',
 'skills': ['Python',
  'PySpark',
  'Great Expectations',
  'Data Plane/ Pipe Dream',
  'Elixir',
  'Erlang',
  'Ruby',
  'Functional programming',
  'Extreme programming practices',
  'Test Driven Design'],
 'description': 'The world is facing a climate emergency. At Maersk we see it as our obligation – as an industry leader in logistics and with the resources available to us – to do all we can to get to net zero operations as fast as possible and to help our customers decarbonize their global supply chains end to end. Our Energy Transition technology team is at the heart of this work, using Advanced Technology and Development Techniques to rapidly develop and continuously deploy the systems we need to identify, track and reduce our emissions.'}

In [116]:
type(json_res)

dict

In [15]:
import pandas as pd
df=pd.read_csv("skills.csv")
df

,TechStack,Expertise
0,Selenium+Java,10+ Years Extensive experience in Selenium wi...
1,"API Testing,Postman,SoapUI","Strong experience of 8+ years in API testing, ..."
2,API Automation,8+ years of extensive experience in API automa...
3,"Mobile Testing,Mobile Automation,Appium","Developed Appium test scripts with Java, handl..."
4,"DevOps,Cloud, AWS,Azure",Worked on CI/CD pipeline integration for test ...
5,"CI/CD,Jenkins,Gitlab,Git",Worked on configuring automated test execution...
6,Python and Pytest,Experience in Python and Pytest for test auto...
7,"SQL,MySQL,POSTGreSQL","Experience in writing complex queries, perform..."
8,"AI,LLM",I have recent experience in working with Large...


In [16]:
import uuid
import chromadb
client=chromadb.PersistentClient('vectorstoreNew')
collection=client.get_or_create_collection(name='portfolio')

if not collection.count():
    for _,row in df.iterrows():
        collection.add(documents=row["TechStack"],
                       metadatas={"Expertise":row["Expertise"]},
                       ids=[str(uuid.uuid4())])

In [17]:
job=json_res
job['skills']

['Python',
 'PySpark',
 'Great Expectations',
 'Data Plane/ Pipe Dream',
 'Elixir',
 'Erlang',
 'Ruby',
 'Functional programming',
 'Extreme programming practices',
 'Test Driven Design']

In [18]:
links=collection.query(query_texts=job['skills'],n_results=1).get('metadatas')
links

[[{'Expertise': 'Experience in  Python and Pytest for test automation, specializing in API and backend testing. My  expertise includes writing scalable test scripts, implementing fixtures, parameterization, integrating with CI/CD pipelines'}],
 [{'Expertise': 'Experience in  Python and Pytest for test automation, specializing in API and backend testing. My  expertise includes writing scalable test scripts, implementing fixtures, parameterization, integrating with CI/CD pipelines'}],
 [{'Expertise': 'I have recent experience in working with Large Language Models (LLMs) to create apps for automated email generation. This includes designing and implementing solutions that use LLMs to generate contextually relevant emails'}],
 [{'Expertise': 'Experience in writing complex queries, performing database validation, data manipulation, joins, indexing, and optimizing queries for test automation.Worked on database testing to validate backend data integrity and ensure seamless application perform

In [19]:

prompt_email=PromptTemplate.from_template(
        """
        ### JOB Description:
        {job_description}
        
        ###INSTRUCTION:
        You are Sushree, a test automation engineer currently working with Boeing.You are a result driven automation tester with 11+ years of experience in
        frontend and backend test automation,mobile,performance and api testing.Skilled in designing and implementing test automation
        stratergies across web,mobile and cloud based application.Currently enhancing your skills in LLM.
        Your job is to write a cold email to the client regarding the job mentioned above in fulfiling their needs for automation tester recruitement.
        Also add the most relevent once from the following expertise to showcase your experience: {links}
        Remember you are Sushree, Senior Automation Test Engineer at Boeing.
        Do not provide a preamble
        ### EMAIL (NO PREAMBLE)
            """
)
chain_email=prompt_email|llm
res=chain_email.invoke({"job_description":str(job),"links":links})
print(res.content)

Subject: Expert Automation Tester for Energy Transition Technology Team

Dear Hiring Manager,

I am excited to apply for the Software Engineer - Python/PySpark/Data Engineer position at Maersk, where I can leverage my 11+ years of experience in test automation to contribute to the company's net-zero operations goal. With a strong background in designing and implementing test automation strategies across web, mobile, and cloud-based applications, I am confident in my ability to support the Energy Transition technology team.

My expertise includes experience in Python and Pytest for test automation, specializing in API and backend testing. I have written scalable test scripts, implemented fixtures, parameterization, and integrated with CI/CD pipelines. Additionally, I have recent experience working with Large Language Models (LLMs) to create apps for automated email generation, which can be applied to enhance the team's productivity.

I am particularly drawn to this role because of the e